In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 26 06:14:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks')
from transformer import Transformer

In [ ]:
START_TOKEN = '<SOE>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<EOE>'

english_vocab = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]
spanish_vocab = [START_TOKEN, ' ','¡', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '¿','?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n','ñ', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z','á','é','í','ó','ú',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

ind_to_esp = {k:v for k,v in enumerate(spanish_vocab)}
esp_to_ind = {v:k for k,v in enumerate(spanish_vocab)}
ind_to_eng = {k:v for k,v in enumerate(english_vocab)}
eng_to_ind = {v:k for k,v in enumerate(english_vocab)}

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset=load_dataset('Helsinki-NLP/opus-100','en-es')
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'translation': {'en': "It was the asbestos in here, that's what did it!",
  'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}}

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, spanish_sentences):
        self.english_sentences = english_sentences
        self.spanish_sentences = spanish_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.spanish_sentences[idx]

In [ ]:
#Preparación del dataset
english_sentences=[]
spanish_sentences=[]
for i in range(len(train_dataset)):
  row=train_dataset[i]['translation']
  english_sentences.append(row['en'].lower())
  spanish_sentences.append(row['es'].lower())

In [ ]:
import numpy as np
PERCENTIL = 97
print( f"{PERCENTIL}th percentile length Spanish: {np.percentile([len(x) for x in spanish_sentences], PERCENTIL)}" )
print( f"{PERCENTIL}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTIL)}" )

97th percentile length Spanish: 291.0
97th percentile length English: 262.0


In [ ]:
max_sequence_length = 300

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
          return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1)


In [ ]:
valid_sentence_indicies = []
for index in range(len(spanish_sentences)):
    spanish_sentence, english_sentence = spanish_sentences[index], english_sentences[index]
    if is_valid_length(spanish_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(english_sentence, english_vocab) \
      and is_valid_tokens(spanish_sentence, spanish_vocab):
        valid_sentence_indicies.append(index)
    elif index<100:
      # print(is_valid_tokens(spanish_sentence, spanish_vocab))
      continue

print(f"Number of sentences: {len(spanish_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 1000000
Number of valid sentences: 913814


In [ ]:
spanish_sentences_exp = [spanish_sentences[i] for i in valid_sentence_indicies][:200000]
english_sentences_exp = [english_sentences[i] for i in valid_sentence_indicies][:200000]
print(len(spanish_sentences))
dataset = TextDataset(english_sentences_exp, spanish_sentences_exp)
print(len(dataset))
print(dataset[1])

1000000
200000
("i'm out of here.", 'me voy de aquí.')


In [ ]:
import torch
import numpy as np
NEG_INFTY = -1e9

def create_masks(eng_batch, esp_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, esp_sentence_length = len(eng_batch[idx]), len(esp_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      esp_chars_to_padding_mask = np.arange(esp_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, esp_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, esp_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, esp_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
import time
from datetime import datetime
from torch import nn
def train_transformer(transformer,train_loader,num_epochs=10,transformer_name=""):
  criterian = nn.CrossEntropyLoss(ignore_index=esp_to_ind[PADDING_TOKEN],
                                reduction='none')
  for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

  optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
  transformer.train()
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  transformer.to(device)

  total_loss = 0

  start_time = time.perf_counter()

  for epoch in range(num_epochs):
      print(f"Epoch {epoch}")
      iterator=iter(train_loader)
      for batch_num, batch in enumerate(iterator):
          transformer.train()
          eng_batch, esp_batch = batch
          encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, esp_batch)
          optim.zero_grad()
          esp_predictions = transformer(eng_batch,
                                      esp_batch,
                                      encoder_self_attention_mask.to(device),
                                      decoder_self_attention_mask.to(device),
                                      decoder_cross_attention_mask.to(device),
                                      enc_start_token=False,
                                      enc_end_token=False,
                                      dec_start_token=True,
                                      dec_end_token=True)
          labels = transformer.decoder.sentence_embedding.batch_tokenize(esp_batch, start_token=False, end_token=True)
          loss = criterian(
              esp_predictions.view(-1, esp_vocab_size).to(device),
              labels.view(-1).to(device)
          ).to(device)
          valid_indicies = torch.where(labels.view(-1) == esp_to_ind[PADDING_TOKEN], False, True)
          loss = loss.sum() / valid_indicies.sum()
          loss.backward()
          optim.step()
          #train_losses.append(loss.item())
          if batch_num % 100 == 0:
              print(f"Iteration {batch_num} : {loss.item()}")
              print(f"English: {eng_batch[0]}")
              print(f"Spanish Translation: {esp_batch[0]}")
              esp_sentence_predicted = torch.argmax(esp_predictions[0], axis=1)
              predicted_sentence = ""
              for idx in esp_sentence_predicted:
                if idx == esp_to_ind[END_TOKEN]:
                  break
                predicted_sentence += ind_to_esp[idx.item()]
              print(f"Spanish Prediction: {predicted_sentence}")


              transformer.eval()
              esp_sentence = ("",)
              eng_sentence = ("somewhere in la mancha, in a place whose name i do not care to remember.",)
              for word_counter in range(max_sequence_length):
                  encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, esp_sentence)
                  predictions = transformer(eng_sentence,
                                            esp_sentence,
                                            encoder_self_attention_mask.to(device),
                                            decoder_self_attention_mask.to(device),
                                            decoder_cross_attention_mask.to(device),
                                            enc_start_token=False,
                                            enc_end_token=False,
                                            dec_start_token=True,
                                            dec_end_token=False)
                  next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                  next_token_index = torch.argmax(next_token_prob_distribution).item()
                  next_token = ind_to_esp[next_token_index]
                  esp_sentence = (esp_sentence[0] + next_token, )
                  if next_token == END_TOKEN:
                    break

              print(f"Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : {esp_sentence}")
              print("-------------------------------------------")

      checkpoint={'model':transformer.state_dict(),'optimizer':optim.state_dict()}
      torch.save(checkpoint,f'./drive/MyDrive/ColabNotebooks/Models/Checkpoints/{transformer_name}_{epoch}')
      print('---------------------- CHECKPOINT SAVED ----------------------')
  end_time=time.perf_counter()
  total_time=end_time-start_time

  print('Tiempo total de entrenamiento: ', total_time)

  return transformer, total_time



In [ ]:
def translate(transformer,eng_sentence):
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  transformer.eval()
  eng_sentence = (eng_sentence,)
  esp_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, esp_sentence)
    predictions = transformer(eng_sentence,
                              esp_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = ind_to_esp[next_token_index]
    esp_sentence = (esp_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  if END_TOKEN in esp_sentence[0]:
    return esp_sentence[0][:-5]
  return esp_sentence[0]

In [ ]:
d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads_1 = 16
num_heads_2 = 8
drop_prob = 0.1
num_layers_1 = 1
num_layers_2 = 2
max_sequence_length = 300
esp_vocab_size = len(spanish_vocab)
train_loader = DataLoader(dataset, batch_size)

transformer_base = Transformer(d_model,
                          ffn_hidden,
                          num_heads_2,
                          drop_prob,
                          num_layers_1,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

transformer1 = Transformer(d_model,
                          ffn_hidden,
                          num_heads_1,
                          drop_prob,
                          num_layers_1,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer2 = Transformer(d_model,
                          ffn_hidden,
                          num_heads_2,
                          drop_prob,
                          num_layers_2,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [ ]:
transformer_base,train_time_base=train_transformer(transformer=transformer_base,train_loader=train_loader)

In [ ]:
torch.save(transformer_base.state_dict(),'./drive/MyDrive/ColabNotebooks/Models/transformer_base')

In [ ]:
transformer1,train_time1=train_transformer(transformer=transformer1,train_loader=train_loader,num_epochs=10)


Epoch 0
Iteration 0 : 4.959647178649902
English: it was the asbestos in here, that's what did it!
Spanish Translation: fueron los asbestos aquí. ¡eso es lo que ocurrió!
Spanish Prediction: ¿¿<SOE>5xxp<SOE><SOE><SOE><SOE><SOE><SOE>,f~,~,rú¡úúú¿:¿{ú~-fe~ee:|~:~~|ef~:~¿rrá<SOE>~<SOE><SOE><SOE>~<SOE>á:eerp<SOE>:=e=ú=7á`=¡===á==fáp,7==áe=pñá`:áááááááá5á=#á=5á55áááá=r<SOE> r7 =:áqrr~~~ú :  r e ñ ñá~~áúá~  á<SOE> <SOE>úáv><SOE><SOE>ñáñ¡ñ =: ácú: :        p <PAD><PAD>  ú ú úú >>5>~ú  ámm55mmmmmáá¡55ú5ú5úííám<SOE>=í<SOE><SOE><SOE><SOE>rrrreá<SOE><SOE>e7<SOE>á<SOE>=<SOE>==áñáííííááááíe  <PAD>r<SOE>áú<SOE><SOE><SOE><SOE><SOE>íáááííáí ápmñáá
Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : ('                         eee      eee                       eee    eeeeeee                   e       eee    eee     ee                                              e e                                                                eeeeeeeeeee   eeeeee        e

In [ ]:
torch.save(transformer1.state_dict(),'./drive/MyDrive/ColabNotebooks/Models/transformer1')

In [ ]:
transformer2,train_time2=train_transformer(transformer=transformer2,train_loader=train_loader,num_epochs=10)

Epoch 0
Iteration 0 : 5.042454719543457
English: it was the asbestos in here, that's what did it!
Spanish Translation: fueron los asbestos aquí. ¡eso es lo que ocurrió!
Spanish Prediction: ,pppdpd:ppp/p7//bbbbbb7bpppb//(7pp7777dp7pp@p7rdb''''edz'wwd'e7we%d%pwzw7zpp7dppppwp:::wp8]%wwwp]6pp]ppppwwwwp%wppzpwpp%pww:wp'wwwzwzwzz]zwzw]wwwwz]]tw']wzwwzze]weeepewwww!:wwwt]zwwwwwzwhww2zbz'b]w'wwzzz:ww]wzzzww2wzzb'rzwzwwwwwbb]wwzrzbwbbwwwbwzwwzwb'zwwwwzuwbbbtwwzt.bbwwww]]]]bzwzzbzpwz@wwzww@bbww
Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : ('                                                                                                                                                                                                                                                                                                            ',)
-------------------------------------------
Iteration 100 : 3.13130521774292
English: ... little lamb, l

In [ ]:
torch.save(transformer2.state_dict(),'./drive/MyDrive/ColabNotebooks/Models/transformer2')


In [ ]:
traduccion1=translate(transformer1,"my name is miguel")
traduccion2=translate(transformer2,"my name is miguel")
print(traduccion1)
print(traduccion2)

mi nombre es miguel<EOE>
mi nombre es migo<EOE>


In [ ]:
traduccion1=translate(transformer1,"real madrid is the best football club in the world")
traduccion2=translate(transformer2,"real madrid is the best football club in the world")
print(traduccion1)
print(traduccion2)

real madri es la mejor de la mejor en el mundo<EOE>
real madrid es la mejor de la mundo en el mundo en el mundo<EOE>


In [ ]:
traduccion1=translate(transformer1,"where there's a will, there's a way")
traduccion2=translate(transformer2,"where there's a will, there's a way")
print(traduccion1)
print(traduccion2)

dónde está habla, hay un mujer.<EOE>
dónde está es un poco es un parte.<EOE>


In [ ]:
traduccion1=translate(transformer1,"where can i buy some ice cream?")
traduccion2=translate(transformer2,"where can i buy some ice cream?")
print(traduccion1)
print(traduccion2)

¿dónde puedo puedo de verdad?<EOE>
¿dónde puedo alguna vez crea?<EOE>


In [ ]:
traduccion1=translate(transformer1,"hard work pays off")
traduccion2=translate(transformer2,"hard work pays off")
print(traduccion1)
print(traduccion2)

hard work paysy de trabajo de los trabajo de la camino de la camigo de la camino de la camino de la camigo de la camino de la camino de la camino de la camino de la camigo de la camigo de la camino de la camino de la campo de la camigo de la camino de la camino de la camino de la camigo dentra dentu
hard vuelta pagar de trabajo<EOE>


In [ ]:
traduccion1=translate(transformer1,"artificial intelligence is going to take over the world")
traduccion2=translate(transformer2,"artificial intelligence is going to take over the world")
print(traduccion1)
print(traduccion2)

artificial inteligencia es un mundo de la mundo de mundo<EOE>
artífica inteligencia es de la mundo a la mundo mundo a la mundo<EOE>


In [ ]:
traduccion1=translate(transformer1,"it's nice to meet you")
traduccion2=translate(transformer2,"it's nice to meet you")
print(traduccion1)
print(traduccion2)

es una menos que deber conocerlo.<EOE>
es un poco me dejarlo.<EOE>


In [ ]:
traduccion1=translate(transformer1,"how old are you?")
traduccion2=translate(transformer2,"how old are you?")
print(traduccion1)
print(traduccion2)

¿cómo está bien?<EOE>
¿cómo estás te estás?<EOE>


In [ ]:
traduccion1=translate(transformer1,"never give up, just do it")
traduccion2=translate(transformer2,"never give up, just do it")
print(traduccion1)
print(traduccion2)

nunca de la camino de lo hagas de lo hagas de lo de la camino de la camino de la camino de la camino de la camino de la camino de la camino de la camino de la campo de la campo de la camino de la camino de la campo de la campo de la campo de la campo de la campo de la campo de la campañer.<EOE>
nunca de dar a mí.<EOE>


In [ ]:
errores=0
i=1
while i<101:
  row=test_dataset[i]['translation']
  try:
    if is_valid_tokens(row['en'].lower(),english_vocab) and is_valid_length(row['en'].lower(),300):
      traduccion1=translate(transformer1,row['en'].lower())
      traduccion2=translate(transformer2,row['en'].lower())
      i+=1
      print(f'Sentence {i}: ',row['en'].lower())
      print('Original Translation: ',row['es'].lower())
      print('Transformer 1: ',traduccion1)
      print('Transformer 2: ',traduccion2)
      print('-----------------------------------------------------')
  except:
    errores+=1

'\xa0'
['i', 'f', ' ', 'y', 'o', 'u', 'r', ' ', 'c', 'o', 'u', 'n', 't', 'r', 'y', ' ', 'p', 'r', 'o', 'd', 'u', 'c', 'e', 'd', ' ', 'o', 'd', 's', ' ', 'f', 'o', 'r', ' ', 't', 'h', 'i', 's', ' ', 'p', 'u', 'r', 'p', 'o', 's', 'e', ',', ' ', 'p', 'l', 'e', 'a', 's', 'e', ' ', 'e', 'n', 't', 'e', 'r', ' ', 't', 'h', 'e', ' ', 'a', 'm', 'o', 'u', 'n', 't', ' ', 's', 'o', ' ', 'p', 'r', 'o', 'd', 'u', 'c', 'e', 'd', ' ', 'i', 'n', ' ', 'c', 'o', 'l', 'u', 'm', 'n', ' ', '6', ' ', 'o', 'n', ' ', 'd', 'a', 't', 'a', ' ', 'f', 'o', 'r', 'm', '\xa0', '3', '.', '”']
'♪'
['♪', ' ', 'j', 'u', 'v', 'i', 'e', ' ', 't', 'h', 'e', ' ', 'g', 'r', 'e', 'a', 't', ' ', 'm', 'a', 'n', ',', ' ', 'w', 'h', 'o', ' ', 'e', 'l', 's', 'e', ' ', 'c', 'o', 'u', 'l', 'd', ' ', 'i', 't', ' ', 'b', 'e', ' ', 'b', 'u', 't', ' ', 'i', '?']
Sentence 2:  the home planet is running out.
Original Translation:  el planeta madre se está agotando.
Transformer 1:  la planeta es la planeta de la camino.<EOE>
Transformer 2:  

In [ ]:
#Preparación del dataset para el modelo final
spanish_sentences_final = [spanish_sentences[i] for i in valid_sentence_indicies][:500000]
english_sentences_final = [english_sentences[i] for i in valid_sentence_indicies][:500000]
batch_size=50
dataset_final = TextDataset(english_sentences_final, spanish_sentences_final)
train_loader_final = DataLoader(dataset_final, batch_size)

In [ ]:
d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads_3 = 8
drop_prob = 0.1
num_layers_3 = 4
max_sequence_length = 300
esp_vocab_size = len(spanish_vocab)

transformer_final = Transformer(d_model,
                          ffn_hidden,
                          num_heads_3,
                          drop_prob,
                          num_layers_3,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [ ]:
transformer_final=train_transformer(transformer_final,train_loader_final,num_epochs=10)

Streaming output truncated to the last 5000 lines.
English: so, felicity, i'm going to punch you in, and i'm going to pay you both to stand here and talk until you make it up.
Spanish Translation: así que, felicity te voy a ingresar, les voy a pagar a los dos para que se queden aquí y hablen hasta que se arreglen.
Spanish Prediction: así que, milizita qe voy a drtlesor  yo  doy a cosar a las pes mart que se hueda  e uí e saclam decta que se hcraglo  
Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : ('algunos manos en manos en un mano de la cama de la cama de la cama de la cama de la cama.<EOE>',)
-------------------------------------------
Iteration 6900 : 1.1403765678405762
English: - it was blue and it had rickrack edging...
Spanish Translation: - era azul y con un cuello...
Spanish Prediction: - esa ulul y com ln paarlo ..      
Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : ('alg

In [ ]:
torch.save(transformer_final[0].state_dict(),'./drive/MyDrive/ColabNotebooks/Models/transformer_final')
torch.save(transformer_final[0].state_dict(),'./drive/MyDrive/ColabNotebooks/Models/transformer_final.pth.tar')


In [ ]:
transformer_final

(Transformer(
   (encoder): Encoder(
     (sentence_embedding): SentenceEmbedding(
       (embedding): Embedding(71, 512)
       (position_encoder): PositionalEncoding()
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (layers): SequentialEncoder(
       (0): EncoderLayer(
         (attention): MultiHeadAttention(
           (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
           (linear_layer): Linear(in_features=512, out_features=512, bias=True)
         )
         (norm1): LayerNormalization()
         (dropout1): Dropout(p=0.1, inplace=False)
         (ffn): PositionwiseFeedForward(
           (linear1): Linear(in_features=512, out_features=2048, bias=True)
           (linear2): Linear(in_features=2048, out_features=512, bias=True)
           (relu): ReLU()
           (dropout): Dropout(p=0.1, inplace=False)
         )
         (norm2): LayerNormalization()
         (dropout2): Dropout(p=0.1, inplace=False)
       )
       (1): EncoderLayer(
       

In [ ]:
transformer_final,tiempo_transformer_final=transformer_final

In [ ]:
errores=0
i=1
spanish_translation_transformer_final=[]
prediction_transformer_final=[]
while i<101:
  row=test_dataset[i]['translation']
  try:
    if is_valid_tokens(row['en'].lower(),english_vocab) and is_valid_length(row['en'].lower(),300):
      traduccion_final=translate(transformer_final,row['en'].lower())
      print(f'Sentence {i}: ',row['en'].lower())
      spanish_translation_transformer_final.append(row['es'].lower())
      print('Original Translation: ',row['es'].lower())
      prediction_transformer_final.append(traduccion_final)
      print('Transformer Final: ',traduccion_final)
      print('-----------------------------------------------------')
      i+=1
  except:
    errores+=1

In [ ]:
d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads_1 = 16
num_heads_2 = 8
num_heads_3 = 8
drop_prob = 0.1
num_layers_1 = 1
num_layers_2 = 2
num_layers_3 = 4
max_sequence_length = 300
esp_vocab_size = len(spanish_vocab)

transformer_base=Transformer(d_model,
                          ffn_hidden,
                          num_heads_2,
                          drop_prob,
                          num_layers_1,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer_base.load_state_dict(torch.load('/content/drive/MyDrive/ColabNotebooks/Models/transformer_base',map_location=torch.device('cuda')))
transformer_base.to(torch.device('cuda'))

transformer1=Transformer(d_model,
                          ffn_hidden,
                          num_heads_1,
                          drop_prob,
                          num_layers_1,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer1.load_state_dict(torch.load('./drive/MyDrive/ColabNotebooks/Models/transformer1',map_location=torch.device('cuda')))
transformer1.to(torch.device('cuda'))

transformer2=Transformer(d_model,
                          ffn_hidden,
                          num_heads_2,
                          drop_prob,
                          num_layers_2,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer2.load_state_dict(torch.load('./drive/MyDrive/ColabNotebooks/Models/transformer2',map_location=torch.device('cuda')))
transformer2.to(torch.device('cuda'))

transformer_final = Transformer(d_model,
                          ffn_hidden,
                          num_heads_3,
                          drop_prob,
                          num_layers_3,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer_final.load_state_dict(torch.load('./drive/MyDrive/ColabNotebooks/Models/transformer_final.pth.tar',map_location=torch.device('cuda')))
transformer_final.to(torch.device('cuda'))

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (attention): MultiHeadAt

In [ ]:
transformer3,time_transformer3=train_transformer(transformer=transformer_base,transformer_name="transformer3",train_loader=train_loader_final,num_epochs=10)
torch.save(transformer3.state_dict(),'./drive/MyDrive/ColabNotebooks/Models/transformer3')


Streaming output truncated to the last 5000 lines.
Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : ('algunos en la mancha, en un lugar que nombre nombre no cuidado para recordar a recordar a recordar a recordar a recordar a la mancha, en un place de algunos de la mancha, en un nombre.<EOE>',)
-------------------------------------------
Iteration 12900 : 0.9245601296424866
English: that.. that can't be the real leatherhead.
Spanish Translation: - eso no puede ser el verdadero leatherhead.
Spanish Prediction: e nso no puede ser ll terdadero.daalheraaad.
Evaluation translation (somewhere in la mancha, in a place whose name I do not care to remember.) : ('algunos en la mancha, en un lugar que nombre cuidar a recordar a recordar a recordar a recordar a recordar a la misma.<EOE>',)
-------------------------------------------
Iteration 13000 : 0.913567066192627
English: - signal?
Spanish Translation: - ¿la señal?
Spanish Prediction: - ¿sa si

In [ ]:
spanish_sentences_final_v2 = [spanish_sentences[i] for i in valid_sentence_indicies][500000:750000]
english_sentences_final_v2 = [english_sentences[i] for i in valid_sentence_indicies][500000:750000]
batch_size=50
dataset_final = TextDataset(english_sentences_final_v2, spanish_sentences_final_v2)
train_loader_final = DataLoader(dataset_final, batch_size)
iterator_final = iter(train_loader_final)

In [ ]:
translate(transformer_final,"at a cost to myself and to my husband.")

'en un costo para mí mismo y a mi marido.'

In [ ]:
test_dataset[0]

{'translation': {'en': 'If your country produced ODS for this purpose, please enter the amount so produced in column 6 on Data Form\xa03.”',
  'es': 'Si su país produjo SAO para estos usos, sírvase anotar en la columna 6 del formulario de datos 3 la cantidad correspondiente”.'}}

In [ ]:
results=[]
prediction_transformer_final=[]
num=0
for i in range(len(test_dataset)):
  if i%100==0:
    print(f'Iteration {i}')
  row=test_dataset[i]['translation']
  frase_eng=row['en'].lower()
  esp_translation=row['es'].lower()
  if is_valid_tokens(frase_eng,english_vocab) and is_valid_length(frase_eng,300):
    num+=1
    esp_prediction=translate(transformer_final,frase_eng)
    results.append({
        "eng":frase_eng,
        "esp":esp_translation,
        "pred":esp_prediction
    })
    prediction_transformer_final.append(esp_prediction)
    # print(f'Sentence {num}: ', frase_eng)
    # print(f'Original translation: ',esp_translation)
    # print(f'Predicted translation: ',esp_prediction)
    # print('--------------------------------------------')


Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
Iteration 1000
Iteration 1100
Iteration 1200
Iteration 1300
Iteration 1400
Iteration 1500
Iteration 1600
Iteration 1700
Iteration 1800
Iteration 1900
